In [ ]:
# for basic operations
import numpy as np
import pandas as pd

# for visualizations
import matplotlib.pyplot as plt

# for market basket analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/dataset/Market_Basket_Optimisation.csv', header=None, names=['item_'+str(i) for i in range(1,21)])
data.head()

item_1     item_2      item_3  ...          item_18  item_19    item_20
0         shrimp    almonds     avocado  ...  frozen smoothie  spinach  olive oil
1        burgers  meatballs        eggs  ...              NaN      NaN        NaN
2        chutney        NaN         NaN  ...              NaN      NaN        NaN
3         turkey    avocado         NaN  ...              NaN      NaN        NaN
4  mineral water       milk  energy bar  ...              NaN      NaN        NaN

[5 rows x 20 columns]

In [ ]:
# size of data
data.shape

(7501, 20)

In [ ]:
# Counts of the number of baskets having 'n' items, (1 <= n <= 20)
basket_sizes = data.notna().apply(sum, axis=1).value_counts()
basket_sizes

1     1754
2     1358
3     1044
4      816
5      665
6      495
7      388
8      327
9      259
10     139
11     102
12      67
13      40
14      22
15      17
16       4
19       2
18       1
20       1
dtype: int64

In [ ]:
# Total number of items purchased
items = data.melt().value.dropna().reset_index(drop=True)
print("A total of {:,} items were sold.".format(len(items)))

A total of 29,363 items were sold.


In [ ]:
# print("There were {} unique items:".format(items.nunique()))
items.unique()
items.nunique()

In [ ]:
item_counts = items.value_counts()
item_counts
# item_counts['mineral water']

mineral water    1788
eggs             1348
spaghetti        1306
french fries     1282
chocolate        1230
                 ... 
bramble            14
cream               7
napkins             5
water spray         3
 asparagus          1
Name: value, Length: 120, dtype: int64

In [ ]:
# making each customers shopping items an identical list
trans = []
for i in range(0, 7501):
    trans.append([str(data.values[i,j]) for j in range(0, 20)])

# conveting it into an numpy array
trans = np.array(trans)

# checking the shape of the array
print(trans.shape)

(7501, 20)


In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
data = te.fit_transform(trans)
data = pd.DataFrame(data, columns = te.columns_)

# getting the shape of the data
data.tail()

asparagus  almonds  antioxydant juice  ...   yams  yogurt cake  zucchini
7496       False    False              False  ...  False        False     False
7497       False    False              False  ...  False        False     False
7498       False    False              False  ...  False        False     False
7499       False    False              False  ...  False        False     False
7500       False    False              False  ...  False         True     False

[5 rows x 121 columns]

In [ ]:
# let's check the columns

data.columns

Index([' asparagus', 'almonds', 'antioxydant juice', 'asparagus', 'avocado',
       'babies food', 'bacon', 'barbecue sauce', 'black tea', 'blueberries',
       ...
       'turkey', 'vegetables mix', 'water spray', 'white wine',
       'whole weat flour', 'whole wheat pasta', 'whole wheat rice', 'yams',
       'yogurt cake', 'zucchini'],
      dtype='object', length=121)

In [ ]:
data.head()

asparagus  almonds  antioxydant juice  ...   yams  yogurt cake  zucchini
0       False     True               True  ...   True        False     False
1       False    False              False  ...  False        False     False
2       False    False              False  ...  False        False     False
3       False    False              False  ...  False        False     False
4       False    False              False  ...  False        False     False

[5 rows x 120 columns]

In [ ]:
data = data.drop(columns=['nan'])

In [ ]:
from mlxtend.frequent_patterns import apriori

#Now, let us return the items and itemsets with at least 5% support:
apriori(data, min_support = 0.01, use_colnames = True)

support                                 itemsets
0    0.020397                                (almonds)
1    0.033329                                (avocado)
2    0.010799                         (barbecue sauce)
3    0.014265                              (black tea)
4    0.011465                             (body spray)
..        ...                                      ...
252  0.011065       (ground beef, milk, mineral water)
253  0.017064  (spaghetti, ground beef, mineral water)
254  0.015731         (spaghetti, milk, mineral water)
255  0.010265    (spaghetti, olive oil, mineral water)
256  0.011465     (spaghetti, pancakes, mineral water)

[257 rows x 2 columns]

In [ ]:
frequent_itemsets = apriori(data, min_support = 0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

In [ ]:
# getting th item sets with length = 2 and support more han 10%

frequent_itemsets[ (frequent_itemsets['length'] == 3) &
                   (frequent_itemsets['support'] >= 0.01) ]

support                                       itemsets  length
240  0.013465               (eggs, chocolate, mineral water)       3
241  0.010532                   (spaghetti, eggs, chocolate)       3
242  0.010932        (ground beef, chocolate, mineral water)       3
243  0.013998               (chocolate, milk, mineral water)       3
244  0.010932                   (spaghetti, chocolate, milk)       3
245  0.015865          (spaghetti, chocolate, mineral water)       3
246  0.010132             (ground beef, eggs, mineral water)       3
247  0.013065                    (eggs, milk, mineral water)       3
248  0.014265               (spaghetti, eggs, mineral water)       3
249  0.010132       (spaghetti, french fries, mineral water)       3
250  0.011065       (frozen vegetables, milk, mineral water)       3
251  0.011998  (frozen vegetables, spaghetti, mineral water)       3
252  0.011065             (ground beef, milk, mineral water)       3
253  0.017064        (spaghetti, ground beef, mineral water)       3
254  0.015731               (spaghetti, milk, mineral water)       3
255  0.010265          (spaghetti, olive oil, mineral water)       3
256  0.011465           (spaghetti, pancakes, mineral water)       3

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
rules

antecedents  ... conviction
0                     (avocado)  ...   1.168147
1               (mineral water)  ...   1.016113
2                        (cake)  ...   1.063198
3                     (burgers)  ...   1.058080
4                     (burgers)  ...   1.039630
..                          ...  ...        ...
427  (spaghetti, mineral water)  ...   1.119933
428   (pancakes, mineral water)  ...   1.251198
429                 (spaghetti)  ...   1.034385
430                  (pancakes)  ...   1.069244
431             (mineral water)  ...   1.024059

[432 rows x 9 columns]

In [ ]:
rules[ (rules['confidence'] >= 0.3) ]

antecedents      consequents  ...  leverage  conviction
0                     (avocado)  (mineral water)  ...  0.003654    1.168147
6                     (burgers)           (eggs)  ...  0.013128    1.224818
38                       (cake)  (mineral water)  ...  0.008142    1.151921
44                    (cereals)  (mineral water)  ...  0.004132    1.267198
58                    (chicken)  (mineral water)  ...  0.008497    1.228438
..                          ...              ...  ...       ...         ...
416       (milk, mineral water)      (spaghetti)  ...  0.007375    1.228597
420      (spaghetti, olive oil)  (mineral water)  ...  0.004799    1.378954
422  (olive oil, mineral water)      (spaghetti)  ...  0.005460    1.315071
426       (spaghetti, pancakes)  (mineral water)  ...  0.005459    1.397557
428   (pancakes, mineral water)      (spaghetti)  ...  0.005593    1.251198

[63 rows x 9 columns]